In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
df = pd.read_csv("./table10.csv", index_col='Year')
df = df.groupby(by=df.index).sum().melt(ignore_index=False)
df = df.rename(columns={'variable':'Type', 'value': 'Number'})
df = df[df.index >= 2009]
display(df)
df_nototal = df[df.Type != 'Total']

natural_df = pd.read_excel("./naturalDisasters.xlsx")[['Year', 'Total Affected']]
natural_df = natural_df.groupby(by=natural_df.Year).sum().reset_index()
natural_df = natural_df[natural_df.Year >= 2009]
display(natural_df)

pdf = pd.read_csv("./policies.csv")
pdf = pdf[pdf.Year >= 2009]
display(pdf)

,Type,Number
Year,,
2009,Total,212788.0
2010,Total,183071.0
2011,Total,187153.0
2012,Total,187081.0
2013,Total,179752.0
...,...,...
2015,Other,16107.0
2016,Other,17299.0
2017,Other,19384.0


,Year,Total Affected
9,2009,3171027.0
10,2010,2237311.0
11,2011,4752034.0
12,2012,1909135.0
13,2013,301014.0
14,2014,3595473.0
15,2015,190827.0
16,2016,608643.0
17,2017,1631881.0
18,2018,4505829.0


,Name,Year,Month,Details
12,Real ID Act,2014,1.0,Broadened grounds for terrorism to not admit a...
14,DACA,2012,8.0,Delayed deportation for millions of illegal im...
15,DAPA,2014,11.0,Delayed deportation for millions of illegal im...
16,Priority Enforcement Program,2015,7.0,Federal government primarily targeted serious ...
17,Executive Order 13769 and 13780,2017,1.0,"AKA Trump travel ban/muslim travel ban, placed..."


In [5]:
natural_df = natural_df[natural_df.Year <= 2016]
df_bush = df_nototal[df_nototal.index < 2017]
fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig = px.area(df_bush, x=df_bush.index, y='Number', color='Type',
#               color_discrete_sequence=px.colors.sequential.Blues,
#               title='LPR admittance under Bush following 9/11 Policies'
#              )
chart_max = (df['Number'].max())

# +(1/12*{month})
#policies = {'USA Patriot Act': 2001+(1/12*10), 'Homeland Security Act of 2002': 2002+(1/12*3), 'Enhanced Border Security and Visa Entry Reform Act': 2002+(1/12*4), }

'''
for policy, year in policies.items():
    fig.add_bar(x=[year], y=[chart_max],
                name=policy, width=0.1,
                hovertemplate=policy,
                marker_color='#b60028'
               )
'''
policies = {'DACA implemented': 2012+(1/12*8)}

for policy, year in policies.items():
    fig.add_bar(x=[year], y=[chart_max],
                name=policy, width=0.1,
                hovertemplate=policy,
                marker_color='#b60028')
    
for i, type in enumerate(df_bush['Type'].unique()):
    fig.add_trace(go.Scatter(
        x=df_bush[df_bush.Type == type].index, y=df_bush[df_bush.Type == type].Number,
        mode='lines',
        name=type,
        line=dict(width=0.5, color=px.colors.sequential.Greens[i]),
        stackgroup='one' # define stack group
    ))

# display(natural_df['Year'])
#fig.add_trace(go.Scatter(x=natural_df.Year, y=natural_df['Total Affected'], name='People affected by natural disasters',),
#              secondary_y=True)

fig.update_layout({
    'title': 'LPR admittance under Obama',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Number of granted LPRs")

fig.update_layout(legend=dict(
    orientation = "h",
    yanchor="top",
    y=-0.25,
    xanchor="left",
    x=0.01
))

fig.show()

In [6]:
fig.write_html("./obama_lpr.txt", include_plotlyjs=False, full_html=False)

In [8]:
lpr_fig = make_subplots(specs=[[{"secondary_y": True}]])


# lpr_fig = px.area(df_nototal, x=df_nototal.index, y='Number', color='Type',
#               color_discrete_sequence=px.colors.sequential.Blues,
#               title='LPR admittance (2000-2019)'
#              )

for i, type in enumerate(df_nototal['Type'].unique()):
    lpr_fig.add_trace(go.Scatter(
        x=df_nototal[df_nototal.Type == type].index, y=df_nototal[df_nototal.Type == type].Number,
        mode='lines',
        name=type,
        line=dict(width=0.5, color=px.colors.sequential.Blues[i]),
        stackgroup='one' # define stack group
    ))


chart_max = (df['Number'].max())
    
for i, row in pdf.iterrows():
    name, year, month, details = row
    lpr_fig.add_bar(x=[year+((1/12) * month)], y=[chart_max],
        name=name, width=0.1,
        hovertemplate=name,
        marker_color='#b60028'
       ) 

lpr_fig.add_trace(go.Scatter(x=natural_df.Year, y=natural_df['Total Affected'],
                             name='People affected by natural disasters',
                             marker=dict(color='#ff7f0e'),
                             visible='legendonly'
                            ),
                             secondary_y=True,
                 )

lpr_fig.add_vrect(x0="2001", x1="2009", 
              annotation_text="Bush", annotation_position="top left",
              fillcolor="#999999", opacity=0.25, line_width=0)

lpr_fig.add_vrect(x0="2009", x1="2017", 
              annotation_text="Obama", annotation_position="top left",
              opacity=0.25, line_width=0)

lpr_fig.add_vrect(x0="2017", x1="2019", 
              annotation_text="Trump", annotation_position="top left",
              fillcolor="#999999", opacity=0.25, line_width=0)

lpr_fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

lpr_fig.show()

In [83]:
lpr_fig.write_html("./lpr.txt", include_plotlyjs=False, full_html=False)